# If we presume that men are from Mars and women are from Venus, are data scientists and data science enthusiast from JupYter? 

<img src=http://cdn.playbuzz.com/cdn/122a43ae-b9f8-429b-bbf0-a0d4a2612b19/260e2af7-d611-444a-8f9a-9922ef7e1f24.jpg>


## Intro


With this wonderful possibility of looking inside the Kaggle community, I wanted to, among other things, check the differences in the male and female survey 
answers and also differences in the approach to basic data science themes. Or are they "the same" regardless of gender.

Certain parts can seem a bit random (hopefully only for now ) and I made a lot of assumptions, first being that we can look at the Kaggle community (members who took part in the survey) as representation of the whole Data Science community.   

This is my first kernel and I'm still a beginner who is tapping in the dark and trying new things, so please be gentle:)   

But of course, advice/comments/ideas are more than welcome.

* [1. Gender differences (general) - multiple choice responses](#rep)
   * [1.1. Average age by country](#age)
   * [1.2. Current job title](#job)
   * [1.3. Used methods at work (wordclouds)](#methods)
   * [1.4. Employment type and education](#edu)
   * [1.5. Importance of University education](#uni)
   * [1.6. Percentage of time spent on certain tasks (gathering data, visualizing,...)](#tasks)
* [2. Gender differences (data scientists) - multiple choice responses](#ds)
   * [2.1. University major](#major)
   * [2.2. Industry they work in](#ind)
   * [2.3. View on algorithm understanding level](#algo)
* [3. Gender differences (general) - free form responses](#free)
   
   


In [ ]:
#import libraries,get used csv files and look at the data
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output
import warnings
warnings.filterwarnings("ignore")

multi = pd.read_csv('../input/multipleChoiceResponses.csv', encoding='ISO-8859-1',low_memory=False)
schema = pd.read_csv('../input/schema.csv', encoding='latin-1',low_memory=False)
freeform = pd.read_csv('../input/freeformResponses.csv', encoding='ISO-8859-1',low_memory=False)



>  <font size=3> Many of the other kernels have shown (and we kind of already knew that) that women are not as represented in this profession as men. For the beginning, a recap of the mentioned.  </font>
<a id="rep"></a>


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 8)
g=sns.countplot(x="GenderSelect", data=multi)
plt.xticks(rotation=90)

gender = pd.DataFrame(multi.GenderSelect)
gender.dropna(inplace = True)
c_gen = gender.GenderSelect.value_counts()
for i in range(0,len(c_gen)):
    print(c_gen.index[i], "\t", str(round(100* c_gen[i] / len(gender), 2)) +"%")





> <font size=3>Also, I guess I'm not going to be very PC in this kernel since I'm ignoring the data from the other groups;).
Let's see what is the average age for both men and women in different countries by using <a href="https://plot.ly/python/">plotly</a> </font>.
<a id="age"></a>


In [ ]:
multi['Country'] = multi['Country'].fillna("Not known")

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

countries = np.unique(multi['Country'])
mean_age_men = []
mean_age_women = []
for country in countries:
    mean_age_men.append(multi[multi['Country'] == country][multi['GenderSelect']=="Male"]['Age'].mean())
    mean_age_women.append(multi[multi['Country'] == country][multi['GenderSelect']=="Female"]['Age'].mean())

data = [ dict(
        type = 'choropleth',
        locations = countries,
        locationmode = 'country names',
        z = mean_age_men,
        text = countries,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Average\nAge (Male)'),
      ) ]    

data2 = [ dict(
        type = 'choropleth',
        locations = countries,
        z = mean_age_women,
        locationmode = 'country names',
        text = countries,        
        marker = dict(
            line = dict(color = 'rgb(0,0,0)', width = 1)),
            colorbar = dict(autotick = True, tickprefix = '', 
            title = 'Average\nAge (Female)')
            )
       ]

layout = dict(
    title = 'Average age of Kaggle members',
    geo = dict(
        scope="world",
        reversescale = True,
        showframe = False,
        showocean = False,
        oceancolor = 'rgb(58,89,213)',
        projection = dict(
        type = 'equirectangular',
            rotation = dict(
                    lon = 60,
                    lat = 10),
        ),
        
        
            ),
        )

fig = dict(data=data, layout=layout)
fig2 = dict(data=data2, layout=layout)
py.iplot(fig, validate=False, filename='worldmap_men_avg_age')
py.iplot(fig2, validate=False, filename='worldmap_women_avg_age')


> <font size=3>So, there's that. Women in this survey are, in average, younger than men. Also, CurrentJobTitleSelect distribution is...</font>
<a id="job"></a>

In [ ]:

pos_m=pd.DataFrame(multi[multi['GenderSelect']=="Male"]['CurrentJobTitleSelect']).dropna()
pos_f=pd.DataFrame(multi[multi['GenderSelect']=="Female"]['CurrentJobTitleSelect']).dropna()

position_m = pos_m.CurrentJobTitleSelect.value_counts()/len(pos_m)*100
position_f = pos_f.CurrentJobTitleSelect.value_counts()/len(pos_f)*100
pose = pd.DataFrame([position_m, position_f])
pose.index = ['Job positions in % (Male)', "Job positions in % (Female)"]
pl=pose.plot(kind='bar',stacked=True, fontsize=20,figsize=(20,18)) 
pl.legend( fontsize = 'x-large')
#print(pose)


> <font size=3>Used methods at work - first for both genders, then separately. For now only with wordclouds because I wanted to try them but I plan to look at this part more closely later</font> ([masking](https://www.kaggle.com/arthurtok/spooky-nlp-and-topic-modelling-tutorial))

<a id="methods"></a>

In [ ]:
mask_f=b'/9j/4AAQSkZJRgABAQEASABIAAD//gA8Q1JFQVRPUjogZ2QtanBlZyB2MS4wICh1c2luZyBJSkcgSlBFRyB2NjIpLCBxdWFsaXR5ID0gMTAwCv/bAEMAAgEBAQEBAgEBAQICAgICBAMCAgICBQQEAwQGBQYGBgUGBgYHCQgGBwkHBgYICwgJCgoKCgoGCAsMCwoMCQoKCv/AAAsIAlUCRgEBEQD/xAAeAAEBAQACAwEBAQAAAAAAAAAACgkGCAQFBwEDAv/EAEwQAAEDAwMCBAMDCAYGCAcAAAABAgMEBQYHCBEJEgoTITEUQVEiYYEVFhcjMmJxkRgkM0KCoUNScnOSkxklNGN0o6SxREVUZJSiw//aAAgBAQAAPwDfwAAAAAAAAAAAAAAA6672erD0++npbpZN0+5awWS7Nh8yDFKKZa68VCKnLe2ip0fM1rvREke1sfr6uRPUyL3ZeNuVstXYtjuzlrmtc5KPJtULov2k9kVbfROTj6/9q/Az4168TT1nNepJYJ929RiFBIqqy3YFZKS1pH/s1DI1qf5zKdbc86i3UC1S726kb5NX76yTnuiuupFznj4+iMfOrUT7kTg+V12X5Zc7i+8XLJ7jUVcn7dVPWyPkd/Fyryp9K0O38b29tV4pb3oNux1CxaWjejoYbTllVHTu4/uvg71ikb9Wva5q/NDcfpVeMM0uuGAUelHVSorhaskt8bY49UMcsi1FHdWIvHfWUVM3vp5kT3dBG+N/qqMi4RrtQtKOsv0pNaqWnqMA6g+lD31TUWCju+Y01sqn8+yfD1ropef3Vbyn0PsVm3LbcsjRFx7X/Ca/u/Z+Dyujl5/h2yKcrtV9sl8h+Ist5pKxnHPfS1DZE/m1VPLAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOpPUb623T76Ydumtu4DVtlwzFKdJaPTvFGtrbzOipyxXxI5GUrHJ6o+ofG1yc9quX0J/OpL4rDqCb0n12BbfLi/RHBJ1dGlJilwc691kf8A39yRGPj5/wBWnbD6KrXOkT1Mw7pdLne7lPeL1cZ6ysqpnS1VVVTOkkmkcvLnuc5VVzlVVVVVeVU/gAAAD/dPUVFJKlRSzvjkav2XxuVqp+KH13RbqC769uT400J3ial4pDEvKUVkzWthpnfc6BsvlvT7nNVDTHp7+MU3kaL3m1YRvwxG36q4k2VkVdklto46DIaWH2WRPL7aarVqevY9kb3qn2puV5KItp+7rbvvf0Utm4PbBqdQZVi90RWsrKNytkppmoivp6iJyI+nnZ3N7o3ojk7kXjhUVfpIAAAAAAAAAAAAAAAAAAAAAAAAAAAOAbmN0u3zZzpJcddNzWq9ow/FrY3+sXO7T9vmSKiq2GGNqK+eZ3C9sUbXPdx6NUnq6rfi9dc9dfylox03LRXacYpJ3wVGoNyYxcguLPZXUzEVzLcxU54civn47XI+F3LTGO/5BfssvlZk+VXysuVyuFS+or7jcKl809TM9Vc+SSR6q573KqqrlVVVV5U8MAAAAAA7fdHPq/a5dJDcVHqDhzqi94HfpYoNQcFfUdsV0pmr6TRc+kVXEiuWOT71Y7lj3IV97Vt02h29HQbHtyW3XNoL9imS0fn0NXF9mSJ6LxJBMxfWKaN6Kx8bvVrmqn0U+hgAAAAAAAAAAAAAAAAAAAAAAAAAAzq6xviMdqfS4grdJMOjg1H1j8j9VhttrUbS2Zzk5ZJc6hvPk+io5KdnMzk7eUja9JCYrfl1Gd3PUk1ck1g3XaqVN7qY3PbZrJTcwWuywuXnyaSmRVbE3hGorl7pH9qLI97vU+GgAAAAAAA7/dBHrZZ30mdf/wAg5xVV130YzKtjZnOOxcyPt8noxt1pGfKeNvCPYn9tG3sX7TYnMrk041GwTV7AbPqlphllDfcdyC3Q19lvNsnSWCsppWo+OVjk9FarVRT3QAAAAAAAAAAAAAAAAAAAAAAAAB415vNnxy0VeQ5DdaahoKGmfUVtdWTtihp4WNVz5HvcqNY1rUVVcqoiIiqpP91vPFiVd0W8bWOldkclPTJ5lHkOs0bVbJL7tfHaGqnLE90+McncvqsLW8MmXBW63W6X26VN8vdyqKytrKh89ZWVczpJZ5XuVz5Hvcqq5znKqq5V5VVVVPHAAAAAAAABsX4XTrmrs+1HpNgO6fMPL0szC58YbfLjUfq8Uu0z/wCzc5y/q6Ooev2v7sUypJ9lskzynEAAAAAAAAAAAAAAAAAAAAAAAAHFdbNbtJduGld71u10z+24vimO0S1V5vl1n8uGnjRURPvc5zlRrWNRXPc5rWornIiy3ddnxG+rfUtu9w277cZrlhuhtLUqySkV6xXDLVY77M9d2r9iDlEcylRVTnh8ivcjGx5fAAAAAAAAAAFL3hZ+ub/SjwOh6dW6rMPM1HxW29un9+uNRzJk1qhZ60r3OXl9ZTRt9/2pYW9y8uike7Z4AAAAAAAAAAAAAAAAAAAAAAAHDtf9f9HdrWj1+1819z6hxnEsaoXVV3vFxk4ZExOERrUTl0kj3K1jI2or3vc1rUVyohJb1yeuhrF1bNWVxrHfj8Y0Zxuvc/D8NfL2yVsictS41/aqtkqXNVe1nKsga5WNVzlkkk6DAAAAAAAAAAA9zp1qHnGkee2bVHTPKa2x5Fj1zhuFkvFumWOejqono+OVjk9nNciL+BYD0JesNhPVp2sR369y0dt1Vw2KCi1Ix2DhrVlc1UjuNO3/AOnqOxyon+jkbJGvKNa9/eIAAAAAAAAAAAAAAAAAAAAAAHr8tyzF8Cxa5Zxm2QUdps1noJq263S41DYYKOmiYr5JpHuVEYxrWq5XKvCIiqSW+IS64WUdVLXX9GWkV0rLfofhNwemLW53dE6/Vbe5jrtUsXheXIrmwxuTmONyqqNfJIhnKAAAAAAAAAAAD790zeoRq90yN3mN7ptJZX1CW+X4XJrA6dWQ3y0yOb8RRSe6J3I1HMcqL2Sxxv4Xs4W0Ha9uW0h3h6AYruX0IyZl2xXL7UyutdUiIj2IvLXwytRV8uaORHxyMX1a9jmr7HPQAAAAAAAAAAAAAAAAAAAAACfbxePWFdcK7/oqtveVL5FM6Cu1iuVBP6SyfZlprP3J7o37E8yf63kM5RWytMDAAAAAAAAAAAAAaz+Fw6z39BfX5Nme4PK/J0m1LurEt1fWzcQ4zfX9scdQrlXhlPUcMilVfRrkikVWtbIrqlQAAAAAAAAAAAAAAAAAAAAAdOuuF1TMa6UuyS7awUclNVZ9kTn2bTSzztR7Z7m+NV+JkZ84KdnMr/k5Ujj5RZWqRt5nmOVaiZfdc/zrIKq7Xu+XGevvF0rplknrKqaRZJZpHL6ue57nOVV91VT1gAAAAAAAAAAAABVP4WDq2Ve+7afNta1tyt1bqjpHRwwfFVk3dUXywKqR01Wqr6ySQrxTyu9V9IHuVXTKaqgAAAAAAAAAAAAAAAAAAAA/FVGoqqqJx9SQHxHPU0XqQ9Qy8yYJkHxem+mvnY3gfky90NWjJP63cW/JfiJm8tcnvDDT8+qKdAgAAAAAAAAAAAAAdh+lRvbvnTz39ab7pKC51MFqst/ip8up6ZXL8XZahUhrYlYn7a+S5z2oqLxJHG73ahbTj2QWPLLBQ5VjF2p6+23Ojiq7dXUkqPiqYJGI+ORjk9HNc1Ucip6Kiop5gAAAAAAAAAAAAAAAAAAABm74nnqZu2AdPmu0709v60uousKVGPYy6CTtmoaDsT8o17eOFarIZGwscio5stVG9P2FJJQAAAAAAAAAAAAAAUX+FE63tr1Fwuz9LPc5kcdPkeP0Sw6RX2rl4S60EaK78kvVf9PAxFWH5Phb5fo6JvmbmgAAAAAAAAAAAAAAAAAAAEfPiQd/6b+ep9l9yxa9/FYXpyq4hh3lSd0UsdLI/wCKqm8ejvOqnTua9P2omwp69qHQkAAAAAAAAAAAAAAHn4vlGR4RktuzPDr7V2q72iuirbXc7fUOinpKiJ6Pjmje1Ucx7XtRyORUVFRFQrk8Pd1n7J1W9tD8b1MraWk1kwClgp84oGI2Nt2hX7EV2gYnCIyRU4lY1OIpeU4aySLnQkAAAAAAAAAAAAAAAAAAA6ade7f83p1dNDOtXMfvSUeY5FT/AJsYCrJO2Rt0rWPak7P3oIGz1KfLmBEX9ojPc5XKrlVVVV9VX5n4AAAAAAAAAAAAAAAfbOnnvp1f6cW7PFd1+jNUrq6w1fZdbRJMrILzbZFRKmhm45+xIxPReF7HtZIidzGqlqm2Dcdpdu8294huY0WvSV+MZpZIblapl472NenD4ZERV7ZY5EfFI3n7L43N+RzwAAAAAAAAAAAAAAAAAAEx3jKd679ZN8+MbOMYvKyWXSPHkqLzBHJ9lb3cWsmejkT0d5dI2k7VX1as0qenK846AAAAAAAAAAAAAAAAG5Xg8eqh+jrUu59MPWPJOyy5dPNd9MJ6uX7NLdWs7qq3oq+zZ42eaxvoiSwyIiK6coyAAAAAAAAAAAAAAAAAAON6x6rYboRpHlGtuolxSksGIY9WXq9VS8fqqWlgfNK5OfdexjuE+akMO53X3Mt1O4vONyWoMquvOc5TXXqvZ3q5sLqiZ0iRMVf7jEcjGp8msRE9jgoAAAAAAAAAAAAAAAB7fAc7zDS3ObNqXp7kNTaL/j11p7lZLrRSds1HVwSNkimYvyc17WuRfqhar0l9/ePdS3Ybg26q3Mpqe8XKiWhzG10zvs2+80ypHVRInu1jnIk0aL6+VNGq+52QAAAAAAAAAAAAAAAAABlN4vfeRUbeOmVDoFjVyWC96z5LFaJEY/tkbaaTtqq17V+ivbSQOT2VlS5P4yuAAAAAAAAAAAAAAAAAGyXg7Ooh+gndzfdh+f33ysa1cg+MxlJ5OI6bIaWJVRrefRvxNM17FX3c+mp2p6qU1AAAAAAAAAAAAAAAAAAlo8YhutXWzqb2/b5Z7l5tq0gw6moZoWv7mtuleja2pei+3PkPoY1T5OhVF9fRMmgAAAAAAAAAAAAAAAAD3mmWo+aaO6j2DVrTi+y2zIMYvNNdbHcqdft0tXTytlikT72vY1fwLitjO6PHN6+z7TjdZizIo6fOcTpLnUUsL+5tJVOZ21NNz68rFO2WJfvjU+rAAAAAAAAAAAAAAAAA9dl2V2DBMUueb5Xco6K1Wa3TV1yrJV4ZBTwxukkkd9zWtVV/gQs7u9wl/wB2O6TUPcxk6SNrM7zG4Xt8MjuVp2VFQ+SOFP3Y2K2NE+SMRD50AAAAAAAAAAAAAAAAACoXwZeuFZqB0yMn0fu1yfLNp/qfWw2+Bz+UgoKymp6pjUT5ItQ6sdx9VVfmpruAAAAAAAAAAAAAAAADo14kHcU7bd0btZL5QXBYLllNnhxS2I1/asq3KdlNUNRfupH1Lv8AAR0AAAAAAAAAAAAAAAAAAG3Xgnt0FDhm5/VnaReqljEzrFKS/wBmWR3HNVbJXxyQsT5ufDXOkX7qVfxpCAAAAAAAAAAAAAAAABhX43XcOto0U0T2p2+u5dfsluGU3WBjuFYyigbS06u+qOdXVHHy5hX6ITrgAAAAAAAAAAAAAAAAAH23pw7vr7sL3yaZ7srI+ZY8PyeCe708C/bqrZJzBXQJ98lLLMxPorkX5FwGLZNj+a4zbsyxK8QXG1XaghrbZcKWTuiqaeViPjlY5Pdrmua5F+aKeeAAAAAAAAAAAAAAAASgeLm3ATaw9YK96eRVSvo9MsLs+PQNa7lnmywrcpXJ+93V6MVf+6RPkZhAAAAAAAAAAAAAAAAAAAqV8Ix1El3SbDanaVnt88/L9EpoqGiSeXmSqx6dXOonpz7+Q5stNwicMjip+V5ehrOAAAAAAAAAAAAAAAARD9WXV/8ATz1NtetVYqrz6a5arXtlul557qOGrkgp1/5MUZ17AAAAAAAAAAAAAAAAAAB3A6FG/a6dPDqWaf6x1FxdFi98r24zncPmdrJLTWyMjkkd9fIkSGpRPmtOieyqWeAAAAAAAAAAAAAAAA4rrrqbQaKaI5lrLdOz4XEcVuN6qe9eG+XS00k7ufu4jUg0ulyr71cqi8XSqfPVVc75qmaReXSSOcrnOX71VVU/gAAAAAAAAAAAAAAAAAAAXN9PLUbKtYdgWhuredVfxF7ynR/GbveJ/X9ZVVNqpppXev1e9y/ifYQAAAAAAAAAAAAAADqj1z8zfgfSA3E3xlQsaz6XXK3dyO45SsYlJx+Pn8fiRYgAAAAAAAAAAAAAAAAAAHuNPcFybVHPrHpnhVvWrvORXimtdppW+81TUStiiYn8Xvan4l2m3rR+zbetAsH0CxypfNb8Hw+2Y/QSyftPhoqWOmY5fvVsaKcwAAAAAAAAAAAAAAAM/PFG5h+aXQ+1kjjl7JrtJYLfAvPv33ygV6fjGyQkGAAAAAAAAAAAAAAAAAAANDPC57X6Lcz1idP6q9UaVFs04oqzNa6JWc/bo0ZHSO5+XbW1FI/n9zj58leAAAAAAAAAAAAAAAAMq/GJZTJj/SDZaWScJfdVLJQvTn9pGxVlTx/OnRfwJVwAAAAAAAAAAAAAAAAAADdrwPWmNJcdXdwWs01Gnn2fG7DZaWoVPXsrKirnlai/xoYVX/CURAAAAAAAAAAAAAAAAxt8bBkKUvTn0yxVH8LXa1U1Uqc+6Q2i5t/950JlQAAAAAAAAAAAAAAAAAACh/wOtNUN0p3EVbo1SKTIccYx/wBXNgr1cn4I5v8AM3dAAAAAAAAAAAAAAABhj44LIvhtAtAsS8zj43ML1V9v18ilp2c/+o/zJ0QAAAAAAAAAAAAAAAAAACn7wYmhVw0/6bOXa1XWN7H6g6l1Lreit+y+ioaeGna9F+a/ELVtX5fYT7zX4AAAAAAAAAAAAAAAE+vjk8kbNkG2rEY5UR1NR5ZWSsRfVUkfaGMVf+U/j8TA0AAAAAAAAAAAAAAAAAAH9rbbbhebjT2i00UtTVVU7IaamgjVz5ZHKjWsa1PVVVVRERPdVLg+mdtKo9iuwjSvalDFG2qxHEaeG9Oiciskuk3NRXSN4/uuqpp3J9zk9z7mAAAAAAAAAAAAAAACZPxrGoiX3qKab6aQVHfFj+kEFVI1Hf2c9Xcq7ub9y+XTwr/ByGN4AAAAAAAAAAAAAAAAAANHvC8dPp+97qZ2PO8tsq1GFaONiyzIHSM5imrY5OLbSqvty6pb5ytVOHR0krfmVvAAAAAAAAAAAAAAAAEbHiIdc7/r31ktcr/e43xR4/lX5s26mc/lIqe2xMo0Vv0R74pJuPrMp0qAAAAAAAAAAAAAAAAAABXh4Zrp5/0DOmdjt4zCx/C5zqsseW5Ws0fbNTwzRp8BRu59U8qm7HqxfVktRMhoYAAAAAAAAAAAAAAAAR5+Jd0Udoj1o9ZaGnpVjoslr6HJKF6px5vx1DBNO7/8lahv+E6HgAAAAAAAAAAAAAAAAAHa3oj7NaDfh1PtKNv+S2ZK7HH378sZdTyN5iktdAx1XPFJ9GzJE2Dn6zp7e5acxjI2JHGxGtanDWtThET6H6AAAAAAAAAAAAAAAATZ+Nq0PXGt4Wj+4enpOyHLtPaqyTSNb6PnttYsquX97y7lG3+DE+hieAAAAAAAAAAAAAAAAAAbN+Cf07tN738ao6mVsPfU4/pStJRdzeUjdV3GlVz0+ju2mVqfc9xTCAAAAAAAAAAAAAAAADDfxv1JTv256D1zoGLLHm12jZIrU7mtdRwqqIvyRVa3lP3U+hOWAAAAAAAAAAAAAAAAAAbp+B6tiS64bgbz2+sGKWKHn/eVNW7/APmUUgAAAAAAAAAAAAAAAAnz8cLq4k+SbftBqOqRFpaG/X+4wc+/mvpKend+Hk1Sfj9xgeAAAAAAAAAAAAAAAAAAbceCJyr4PdtrZhHmcflDTmiruzn3+Hr0j5/D4r/MpEAAAAAAAAAAAAAAAABIR4nrdgu6jrAah01sufxNl02jp8Js/D+UYtEjnVifTlK6asT+CIZ9AAAAAAAAAAAAAAAAAAGt/gxcp/IXVayexvk4betFbtTNYq+72XG1zov8UbE/+alRoAAAAAAAAAAAAAAABwHdTr3ju1nbRn+5LK+x1vwXD7jfKiJ7uPO+Gp3ytiT957moxET1VXIie5CpmuYZFqHmV2z/AC+5PrbtfLnUXC6Vkn7U9TNI6SSRfvc9zl/E9YAAAAAAAAAAAAAAAAAAaEeFs1K/R11tNJqaeo8umyOmvdmqnKvHPmWmqkib9/M0UKfiV6AAAAAAAAAAAAAAAAGaPi09Xblpf0acqx+13J1M/N8vstgkcx3DpIviFrXxov0c2iVF+reU9lUkxAAAAAAAAAAAAAAAAAAB2H6SWpX6IuqDt+z99R5UFHq/YI62Xnjtppq6KCdf+VI8t1AAAAAAAAAAAAAAAAMYPGz32702wfSjGoE/qFZq82pqV5/0sVrrWx//AKzS/wAiaIAAAAAAAAAAAAAAAAAAHs8Lym5YNmNpzazP7ayz3OCupHc8cSxSNkav82oXt4zkNty3G7fldmm8yjudDFV0kn+tFIxHtX+TkPOAAAAAAAAAAAAAAABjr41uyfFdNLTnIGs5Wj1xooVXj2bLZ7sq/wCcbf8AImNAAAAAAAAAAAAAAAAAAALYejDrUm4PpS6A6oSVfxFRNplbLfXzq7lZauhi+Bncv3rNTSKv3qp2bAAAAAAAAAAAAAAABmR4ubTVc76MuS5OkHf+ZmcWG893H7HfUrb+f/XcfiSdAAAAAAAAAAAAAAAAAAAFQ3g1NzCaq9NbIdvFyrUfXaVZ5UxUsCLz5dtuKfGQr93NT8f/AMJrsAAAAAAAAAAAAAAADqX13dNP0s9HrcPivw/mrTaa1t4RnHPrbu24Iv4fC8/gRbgAAAAAAAAAAAAAAAAAAG5fgf77fafcNr1jFPVuS2VeGWeqq4OPR88NXOyF34NnnT/EUYgAAAAAAAAAAAAAAA4rrrpjQa2aI5lo1dXNSly7FbjZalXp6JHVU0kDufu4kUg2u9puNhu1VYrvSOgq6KpfBVQPT7UcjHK1zV+9FRU/A8YAAAAAAAAAAAAAAAAAAFFPgh9BKyy6J647m7jbXJFkOS2vG7VUvbx6UMEtTUI36oq19PyvtzGie6KbpgAAAAAAAAAAAAAAAESfWD0Z/QB1SdfNLIqTyKaj1Ru1VboOOPLo6qodV07fwhnj/idbwAAAAAAAAAAAAAAAAAD/AHTU1TW1MdHR075ppnoyKKJiuc9yrwjURPVVVfTgtY6MuySfp8dNnTDbVfKNsWRUVlW55hw1OfytWvdVVMaqn7XlOl8hHfNsDTtCAAAAAAAAAAAAAAAAS3eMf22/oo6ntp13tlv7KDVTAaKsqahG8JJcaFXUMzfv4p46Fef3zJQAAAAAAAAAAAAAAAAAA1d8Kz0lrtvQ3d0u8TVzEHyaW6SXFlXTPq4f1F7yFnbJS0rUX0kZAqtqZeOURWwscipKpU+AAAAAAAAAAAAAAAADIfxjuzm465dPXHdz+MW9Z7jo3lHnXFGx9zm2i5eVTVDk49fs1DKFy/JGI9y/skvYAAAAAAAAAAAAAAAAAO5HQ86U9R1cN5SaE3fUD82sVx2yPv8Amdxpka6tfQRzxQ+RSMcit86SSZje9yKyNqueqPVGxvr520badFNoGh+P7dNvGC02OYjjNH8ParZTKruOXK58kj3KrpZXvc575HKrnOcqqqqpzsAAAAAAAAAAAAAAAAHEdfdFcI3I6HZft+1KofibBmuN1tlu8SInd5FTC6J7m8+z2o/ua73RyIqeqERG93Z7q5sK3R5ftW1qtUlPecUuslPHV+Q5kVypFXmnrYefeKaJWyNX3RHdq8OaqJ8oAAAAAAAAAAAAAAAAAPtfT635a59NzdHYN1OgNwhS62hXwXG1VquWkvFBLwk9FUNaqK6N6IioqLyx7GPbw5jVStzpbdZrZv1WtO4rtopmEdozajoWy5PpteqhrLpbH8Ij3RovCVdOjlREqIkVvDmo9I3r2J21AAAAAAAAAAAAAAAAAM++v70UcR6r+3n85tO6Git2tOE0UsmD3qXtjbdIfV77TUvX08qReVje7+xld3co18qOkYy7E8mwLK7ngua2Kqtd5stwmoLtbK2FY5qSphescsMjF9Wva9rmqi+qKioeuAAAAAAAAAAAAAAAAAPa4Nneb6Y5db8/02zG6Y/fbVUJUWu9WS4SUtXRyp7SRSxOa+NyfVqopr50/vGM7wNCaG36fb3tNqHV2w0zWwrk1DM225BHGnp3SORq09YrWonCOZE9y8q+VVXk2q2I9dzpk9QpKOyaLbiKGz5VV9rW4Lm/bart5i/6ONkrljqnf+GklRPqdwQAAAAAAAAAAAAAAADimtuumjm27TS56x696l2bEcXs8PmXG932uZTwRJ8m9zl+09y+jWN5c9yo1qKqohGv1qd2O3be/wBSnUncztcxGstWJZHW03w8tbAkD7nPDTRQTV/konMKTvjWTtd9te7vf2ve5reqwAAAAAAAAAAAAAAAAAARVReUXhU9lO7OxHxB/VE2C1VBaME3A1mYYjRq1q4PqG991oPKT2jie9yVFK1E54SCVjeV5Vq+xu700PFX7Cd7dZQaZ6/quime1XbFFTZNcWSWS4TLwiNguPDGxuVfZlQ2L3RrXyKaiQzQ1ELKinlbJHI1HMex3KORfVFRfmh/oAAAAAAAAAAAAAH86yspLfSS19fVRwQQRuknnmejWRsanKucq+iIiIqqq+xlZ1OvFfbH9mv5R0y2rJBrXn9P3wufZa7sx+3Sp6czVzUX4lUXheymR7XcK1ZY1J3OoH1QN53U11JTUPdZqvUXOCllc6x4rbkdTWezNd6dtNSo5WtXjhFler5Xoid73cIdfQAAAAAAAAAAAAAAAAAAADRDpEeI23idMOroNL8qqqjUvSFkjWS4TfK9yVFpj59XWypcjlp+PfyHI6F3rw1jnLIlNPT56n+zXqb6YpqPtW1TguNRTRMdfcVuKNp7xZXu/uVNMrlVqc8okrFfE9UXse7hTsGAAAAAAAAAAAAfwuVyt1mt893u9whpKSlhdLU1VTKkccMbUVXPc5yojWoiKqqvoiIZbdRvxY3T+2fuuGAbbHv1uzemV0XbjdakNhpJU9P1txVrkm49F4pmStXhUV7FMFOoj10eov1LKmrsmtus01mwyoevlaeYZ32+0IznlGzMRyyVnHvzUPk4X9nt9jp8AAAAAAAAAAAAAAAAAAAAADlWi+uWse3LUa36u6C6nXzD8ntb1dQXzHrlJS1MSL6Ob3xqiqxyejmLy1yKqKioqoanbZvGWdSLSegorBr/AKZ4DqlR0sbWT3GpopbRdanj5umpXLToqp80pvf1NP8AY/4tPpjbpp6LE9bbjdtFMmqlbGsOZo2e0OlX5MuUKdjGp831MdO30NNsZyjGc1x+jy3DcioLvarjA2e33O2VjKinqYnerXxyMVWvavyVFVFPPAAAAAAAAAAM2Oqn4nDY507Kq4aU6aTs1d1PpFfFPjeM3Jjbfapk5Ttrq5Eexj0VFRYYmySorVR6R8opPJ1Get/1Cepvcai268avyWvDXzd9Lp1iPfQ2aJEXlvmRo5X1bkX1R9Q+RUXnt7U9DqKAAAAAAAAAAAAAAAAAAAAAAAAdg9jvVO349OnIG3bajuGvVht76jza7FqmRKuz1y/3vNopu6JXKnp5jUbIiKva9q+puP06vGT7ftWqq3ab9Q7Sx2m95nc2Fc4xdstZYpHr6d81OvdU0bVX09FqGp7ucxOeNmNO9R9PtXcKt2pOlWcWjJcevFOlRar5YrjHV0lXEvs+OWJysen3oq+x7oAAAAAAAA+L74+oFtQ6dGj02te67VSjx+3cPZarc1fNuF4nanPw9HTNXvnk9U544axF7nuY3lyTV9WLxRW8vf2tz0j29TVmkGllSr4X26zV6/lq8wryn9drGcLGxzfeng7WcOVr3zJwpl+AAAAAAAAAAAAAAAAAAAAAAAAAAdj+nv1X98PTIzZMn2tawVNHaqipbLesLu3NVZbrxwi+dSuVER6onHnRKyVE9EeicoU69H/xBG0bqrWSlwJauHAdXYabuuWnl5rWr8arW8vmts6o1KuPhFcrOEmYiKrmdqJI7vwAAAAAAAZa9abxNW33pzzXTb5tqprbqTrJC18NXTNqO6z4xL7f16WNyLNO1f8A4WNyOThfMfEvCOmY3Y7w9yW+LWKv133SasXPLckrvspU18qJFSQ8qraemhaiR08LVVeI42taiqq8cqqr80AAAAAAAAAAAAAAAAAAAAAAAAAAAPLsN+vmLXujyXGL1V265W+pjqaC4UFS6GemmY5HMkjkYqOY9rkRUcioqKiKhQ90DvFIW3U91l2a9TXNKehyRfLosR1buEjYqe6r6NZTXR68NhqPZG1S8Mk9pO1/25d0mua5qOa5FRU5RUX3P0AAAAA9dluXYrgOL3DN85ySgs1mtNHJV3S7XSrZBTUdPG1XSSyyPVGsY1qKqucqIiJypO51xPFaZPq5+WNqvTCyKusWLO8ykyDVqNr6e4XZvq10dtReH0kCpz/WFRJn8/YSJE7pMOZppaiZ1RUSuke9yue97uVcq+qqqr7qf5AAAAAAAAAAAAAAAAAAAAAAAAAAAABut4bDxGz9NJ7F089/2eK7GpXx0Ommot4qeVtDlVGx2yulev8A2VV4bDO5f1K8RvXyu1YaJUVHIioqLz9D9AAAAOF7hdxOim1LR+9697hdRLdi2J49SrPdLxc5e1jE9msY1OXSyPdw1kbEc97lRrUVVRCVfrmeIU1r6p+UVWjWky3HC9DrdW91Bjay9lZkT2O5ZV3JWKqL6ojmUyKsca8KqyPaj0zfAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKL/Cw9d9+rNmtXTJ3f5or8otlL5Gk2U3Oo5ddqSNvKWiZ7l9aiJifqHKv6yNvl+j42eZuaAAAD5zuw3YaDbJdCL7uP3I55TY9i1gp++pqpl7paiVefLpoI0+1NPI77LI2+qr9ERVSRjrMdaDXvq5a4re8gkqsd0zsFVImCYBHU8x0rF5b8XVdv2Zqx7f2n+qRoqsZ6dzn9MAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAeZj+QX3E79Q5Vi95qrdc7ZWRVduuFDUOinpZ43o+OWN7VRzHtciORyKioqIqFgvQB6s9n6qey6jvuY3SnZqpgrYbRqRbo+1rp5uxfIubGJ7R1LGOcqIiI2VkzETta1V71gAA4VuK3D6PbUNFMi3Da+ZtS49iWLW51Zd7nVr6MYnCNYxqeskr3K1jI2ornve1rUVVRCQLrNdY/XLq37gpcqyGoq7Hptj9VLHp9giT/q6GBfs/FVCNXtlrJWoivf6oxF8ti9qcu6agAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA7WdGHqNZJ0xN++IbgmXSqbiVXUts+ottgVXNrbJUPa2ZVYn7b4VRlRGnoqvga3nhzubRrHfLNk1ko8kx26U9db7hSx1NDW0kqSRVEMjUeyRjk9HNc1UVFT0VFRTygAeLer1Z8bs1XkWQ3WmoLfQUslTXV1ZM2OGnhY1XPke9yojGtaiqrlXhERVUkt8Qv1xMs6pOukukmkF4qaDQ7CLpI3GaFrnRrkVWzujW7VDV4X7SK5II3JzHG5VVEfI9EzfAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABVt4SzfBc91PTJi0azW/urcj0XvP5uKsz+6VbO+NJrc537rWedTM/do0NRgAYReLk6yVZgllm6Ve3nInw3W80MNVq/dqSTtdTUMrUkgtDVT1R0zFZNN7fqnRs5cksjUngAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANZvB07nK/SLqg1+gFXdFZatWMIrKNKNXcNkuNA1a6nk+9zYI65qf75SpYA+b7wdy+F7Ntree7pdQW+ZasFxeru01MkqMdVyRRqsVMxy+iPll8uJvP96RCHvcDrnqJua1wyzcJq1eFr8kzO/1V3vNT6o1Z55FerWIqr2xt5RrG+zWta1PREOHgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAH3Xpi7hP6KfUM0Y3BT13w1HjWotrmu8vdxxb31DYqxOflzTyTJ+JcOAYteNC3nS6Y7Q8F2U4xclZX6n5A67ZFHG/wD+U21WPZE9PpJVywPav/2biagAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAuJ6ZW4T+lX09tGdwc9d8RWZNp1a6i7y93d/1g2nbFVpz8+KiOZvP3H3MEjviot0/wDSW6v+aY7a7j8RZ9L7XRYdbVY/lqSwNdPWensjm1lTURqvuqRN+iImcoAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABVd4P3XB2qPSJp9OqqtV82nGod4sscL3cuZBOsVyYv+yr66VE+9jvoanHpdSM9x7SvTy/an5dVJBacbstVdLpOq/2dPTwulkd+DGOUhA1h1NyLWvVvKdZcvl8y7Zbkddero9Xc91TVVD55V5+f25HHHAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADezwP+t3w2X697b66s5+NttmyW106u/Z8mSopap3H3+fRp/hT6lCB0b8SFuBm279GnWi+W+s8q4ZPZoMVoWo7hZEuVTHS1Def/CyVLv8JHOAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADSvwmWt36IusxiOLzVfk02oOJ3rG6hzncNVfhvj42r/ALUtBE1PvchWiYneNq1w/NrZ9o/t5p6zy5cu1Bqr3NG13rJDbaNYlav7vfco1/ixPoTaAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA7AdKPVb9CPUy0E1PlqfJp7Zq1Yfj5OeO2klrooaj/AMmSQt8JmfGs6sOyTqB6aaPU9WslPiulLK6SNHekVTXXCqR6cfJVipadf4KhjSAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADybNd7jj94pL9aKl0NXQ1MdRSzN945GORzXJ/BURS9PSbP7dqxpXjOqdo7fhMlx+iutL2rynl1EDJm8L/B6EkXigNU3ao9bDV5IZ1fSY5+SLHRoq/sfD2ul81P8Anvn/AJnQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtP6Gup8msHSF285hPMsskOmVvtMkjl5Vzre1aBVVfmvNMvP3knXV3zr9JPVM3D5e2bzIptZcihpZOee6CG4TQRL/AMEbTrqAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAVteE8z388uijp9Y1m73YvkeQWt3K8q3uuc9Wif8NWn4cErG42/zZXuEzvKKmZZJLlmd0qpJFX1c6Srleq/iqnDAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACm/wVGe/lrpx6i6ezT98ti1jqamNqr+xDU2y39qfw74JV/FSZm9XKS83mru83PfV1Mkzufflzlcv/ALnjAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFBPgcMydLY9yGns0/pT1eLXGmj59/MbdIpF/8uL+aE+wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANs/BGZj8DvE1n0/wDO4/KemlLcOzn3+FuEcfP4fF/5mJgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANHPDDbjmbYt+uXZ3LWJC2s0hr6DlVREVXXS1SfP/dGcYAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOb6C6r5Jo9mFTk2LP7aie2vpXr38fYdJG9f82If/9k='
mask_m=b'...'
mask_mf=b'...'
#c

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from scipy.misc import imread
import base64
import codecs

f3 = open("mf.png", "wb")
f3.write(codecs.decode(mask_mf,'base64'))
f3.close()
imgmf = imread("mf.png")
maskmf= imgmf

methods=multi["WorkMethodsSelect"].dropna().values
plt.figure(figsize=(14,12))
wc=WordCloud(relative_scaling = 0.5,max_words=5000,
                      stopwords = STOPWORDS, mask=maskmf,background_color="black", max_font_size= 50).generate(" ".join(methods))
plt.imshow(wc.recolor( colormap= 'Spectral_r' , random_state=17))
plt.title("Methods", fontsize=16)
plt.axis("off")


In [ ]:

f1 = open("male.png", "wb")
f1.write(codecs.decode(mask_m,'base64'))
f1.close()
imgm = imread("male.png")
# img = img.resize((980,1080))
maskm= imgm

f2 = open("fem.png", "wb")
f2.write(codecs.decode(mask_f,'base64'))
f2.close()
imgf = imread("fem.png")
# img = img.resize((980,1080))
maskf= imgf

methods_m = multi[multi['GenderSelect']=="Male"]['WorkMethodsSelect'].dropna().values
methods_f = multi[multi['GenderSelect']=="Female"]['WorkMethodsSelect'].dropna().values



plt.figure(figsize=(8,8))

wc_m=WordCloud(relative_scaling = 1,max_words=5000,
                      stopwords = STOPWORDS, mask=maskm, background_color="black", 
             max_font_size= 50).generate(" ".join(methods_m))
plt.imshow(wc_m.recolor( colormap= 'Spectral_r' , random_state=17))
plt.title("Methods_m", fontsize=16)
plt.axis("off")
plt.show()

plt.figure(figsize=(10,8))

wc_f=WordCloud(relative_scaling = 1,max_words=5000,
                      stopwords = STOPWORDS, mask=maskf, background_color="black", 
             max_font_size= 50).generate(" ".join(methods_f))
plt.imshow(wc_f.recolor( colormap= 'Spectral_r' , random_state=17))
plt.title("Methods_f", fontsize=16)
plt.axis("off")
plt.show()



> <font size=3>Comparing male to female employment type and education.</font>
<a id="edu"></a>

In [ ]:

employ_m = pd.DataFrame(multi[multi['GenderSelect']=="Male"]['EmploymentStatus'])
employ_f = pd.DataFrame(multi[multi['GenderSelect']=="Female"]['EmploymentStatus'])

brm = employ_m.EmploymentStatus.value_counts()
brf = employ_f.EmploymentStatus.value_counts()

em = pd.DataFrame([brm, brf])
em.index = ['Male_employment', "Female_employment"]
print(em)



In [ ]:
formal_m = pd.DataFrame(multi[multi['GenderSelect']=="Male"]['FormalEducation'])
formal_f = pd.DataFrame(multi[multi['GenderSelect']=="Female"]['FormalEducation'])
c_formal_m= formal_m.FormalEducation.value_counts()#[:10]
c_formal_f= formal_f.FormalEducation.value_counts()#[:10]
for i in range(0,len(c_formal_m)):
    print("Men: ", c_formal_m.index[i], "\t", str(round(100* c_formal_m[i] / len(formal_m), 2)) +"%")
    print("Women: ",c_formal_f.index[i], "\t", str(round(100* c_formal_f[i] / len(formal_f), 2)) +"%")

> <font size=3>What's the general view on importance of University education when it comes to analyzing data</font>
<a id="uni"></a>

In [ ]:

uni_m=pd.DataFrame(multi[multi['GenderSelect']=="Male"]['UniversityImportance']).dropna()
uni_f=pd.DataFrame(multi[multi['GenderSelect']=="Female"]['UniversityImportance']).dropna()

university_m = uni_m.UniversityImportance.value_counts()/len(uni_m)*100
university_f = uni_f.UniversityImportance.value_counts()/len(uni_f)*100
uni = pd.DataFrame([university_m, university_f])

uni.index = ['UniversityImportance % (Male)', "UniversityImportance % (Female)"]
pll=uni.plot(kind='bar',stacked=True, fontsize=20,figsize=(20,18)) 
pll.legend( fontsize = 'x-large')



> <font size=3> I guess we can connect female opinion about the uni importance with the actual formal education. 
Also, how much time do they spend for certain tasks.</font>
<a id="tasks"></a>

In [ ]:
perc={}
tasks=['TimeGatheringData','TimeModelBuilding','TimeProduction','TimeVisualizing','TimeFindingInsights',"TimeOtherSelect"]
for task in tasks : 
    perc[task]={ 'Female':multi[multi['GenderSelect']=='Female'][task].mean(),
    'Male':multi[multi['GenderSelect']=='Male'][task].mean()}                  
x=pd.DataFrame(perc)    
x.transpose().plot(kind='barh',fontsize=20,figsize=(20,18))
plt.xlabel("Time spent on a task (%)",fontsize = 20)
plt.legend(fontsize = 25)


> <font size=3>Let's get more concrete and look at the formal education of those who said they currently work as data scientists - from what fields do the come from (major) and in what industry do they work</font>
<a id="major"></a>

In [ ]:
multi.CurrentJobTitleSelect.unique()

df_formal_m=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Male"]["FormalEducation"])
df_formal_f=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Female"]["FormalEducation"])
ds_formal_m=df_formal_m.FormalEducation.value_counts()
ds_formal_f=df_formal_f.FormalEducation.value_counts()

ds_formal = pd.DataFrame([ds_formal_m, ds_formal_f])
ds_formal.index = ['Male Data Scientists', "Female Data Scientists"]
pl=ds_formal.plot(kind='bar',stacked=True, fontsize=20,figsize=(20,18)) 
pl.legend(loc=1, fontsize = 'xx-large')


for i in range(0,len(ds_formal_m)-1):
    print("Men: ", ds_formal_m.index[i], "\t", str(round(100* ds_formal_m[i] / len(df_formal_m), 2)) +"%")
    print("Women: ",ds_formal_f.index[i], "\t", str(round(100* ds_formal_f[i] / len(df_formal_f), 2)) +"%")

In [ ]:
#print(multi.MajorSelect.unique())
order=['Management information systems', 'Computer Science',
 'Engineering (non-computer focused)', 'Mathematics or statistics',
 'Electrical Engineering', 'Other', 'Biology', 'Physics',
 'Information technology, networking, or system administration',
 'A social science', 'A health science', 'A humanities discipline',
 'Fine arts or performing arts', 'I never declared a major', 'Psychology']


df_major_m=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Male"]["MajorSelect"])
df_major_f=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Female"]["MajorSelect"])
ds_major_m=df_major_m.MajorSelect.value_counts()
ds_major_f=df_major_f.MajorSelect.value_counts()

multi.MajorSelect.unique()
fig, ax =plt.subplots(1,2)
fig.set_size_inches(14, 12)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=90)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=90)
ax[0].set_title("Male Data Scientists - major select")
ax[1].set_title("Female Data Scientists - major select")
sns.countplot(x="MajorSelect", data=df_major_m, ax=ax[0],order=order)
sns.countplot(x="MajorSelect", data=df_major_f, ax=ax[1],order=order)

fig.show()



> <font size=3> Majority of data scientists come from CS and maths or statistics. Interestigly, third most common major selection for women in DS is physics, while for men it's engineering (with physics coming close).</font>

<a id="ind"></a>

In [ ]:
#print(multi.EmployerIndustry.unique())
order=['Internet-based', 'Mix of fields', 'Technology', 'Academic','Government',
 'Non-profit' ,'Financial' ,'Retail', 'Telecommunications' ,'Other',
 'Pharmaceutical', 'Military/Security' ,'CRM/Marketing', 'Manufacturing',
 'Hospitality/Entertainment/Sports', 'Insurance']
df_ind_m=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Male"]["EmployerIndustry"])
df_ind_f=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Female"]["EmployerIndustry"])
ds_ind_m=df_ind_m.EmployerIndustry.value_counts()
ds_ind_f=df_ind_f.EmployerIndustry.value_counts()


fig, ax =plt.subplots(1,2)
fig.set_size_inches(14, 12)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=90)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=90)
ax[0].set_title("Male Data Scientists - employer industry")
ax[1].set_title("Female Data Scientists - employer industry")
sns.countplot(x="EmployerIndustry", data=df_ind_m, ax=ax[0],order=order)
sns.countplot(x="EmployerIndustry", data=df_ind_f, ax=ax[1],order=order)

fig.show()



> <font size=3>Is there a difference in viewing their own understanding of algorithms? </font>
<a id="algo"></a>

In [ ]:
#print(multi.AlgorithmUnderstandingLevel.unique())
order=['Enough to explain the algorithm to someone non-technical',
 'Enough to refine and innovate on the algorithm',
 'Enough to tune the parameters properly',
 'Enough to code it again from scratch, albeit it may run slowly',
 'Enough to run the code / standard library',
 'Enough to code it from scratch and it will run blazingly fast and be super efficient']
search_m=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Male"]["AlgorithmUnderstandingLevel"]).dropna()
search_f=pd.DataFrame(multi[multi['CurrentJobTitleSelect'] == "Data Scientist"][multi['GenderSelect']=="Female"]["AlgorithmUnderstandingLevel"]).dropna()
ds_search_m=search_m.AlgorithmUnderstandingLevel.value_counts()
ds_search_f=search_f.AlgorithmUnderstandingLevel.value_counts()


fig, ax =plt.subplots(1,2)
fig.set_size_inches(14, 12)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=90)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=90)
ax[0].set_title("Male Data Scientists - AlgorithmUnderstandingLevel")
ax[1].set_title("Female Data Scientists - AlgorithmUnderstandingLevel")
sns.countplot(x="AlgorithmUnderstandingLevel", data=search_m, ax=ax[0],order=order)
sns.countplot(x="AlgorithmUnderstandingLevel", data=search_f, ax=ax[1],order=order)

fig.show()


> <font size=3>Let us look at free form responses for a bit. First, which questions have the most answers. </font>

<a id="free"></a>

In [ ]:
freeform.count().sort_values(ascending=False).head(10)

In [ ]:
#freeform["PastJobTitlesFreeForm"].unique()




![](https://altopedia.com/images/5/5b/Work-in-progress.gif)